# 02 — RAII & Smart Pointers: The End of malloc 🗑️

## 1. The Core Philosophy: RAII

RAII stands for Resource Acquisition Is Initialization.
It's a terrible name for a beautiful concept.

The Rule: 
1. Acquire resources (memory, file handles, mutexes) in the Constructor.
2. Release resources in the Destructor.

Because C++ guarantees destructors are called when a variable goes out of scope (even if an error occurs), resource leaks become impossible if you follow this pattern.

### The C Problem
```c
void c_risky_code() {
    int* ptr = malloc(sizeof(int));
    if (error_condition) {
        return; // LEAK! Forgot to free(ptr)
    }
    free(ptr);
}
```

### The C++ Solution (Smart Pointers)
We don't manage pointers manually. We wrap them in objects that do it for us.


---

## 2. std::unique_ptr (Your new Default)

std::unique_ptr is a wrapper around a raw pointer.
* Overhead: Zero. It compiles down to the exact same assembly as a raw pointer.
* Semantics: Exclusive ownership. Only one unique_ptr can hold the resource at a time.
* Cleanup: Automatically calls delete when it goes out of scope.

Note: To use this, you need #include <memory>.


In [1]:
#include <iostream>
#include <memory> // Required for smart pointers
#include <string>

// A noisy class to help us visualize memory management
class Tracker {
    std::string name;
public:
    Tracker(std::string n) : name(n) {
        std::cout << "[Constructed] " << name << std::endl;
    }
    ~Tracker() {
        std::cout << "[Destructed ] " << name << " (Memory Freed)" << std::endl;
    }
    void say_hello() {
        std::cout << "Hello from " << name << std::endl;
    }
};


### ⚠️ NOTE: Notebook Scopes
In standard C++, destructors run when the function returns.
In a Notebook, variables stay alive until you restart the kernel.

To force the destructor to run now, we wrap our code in { } blocks. This simulates a function scope.


In [2]:
std::cout << "--- Scope Start ---" << std::endl;
{
    // OLD C++ way (Avoid): Tracker* t = new Tracker("Old");
    
    // MODERN way: std::make_unique<Type>(arguments...)
    // This creates the object on the heap and wraps it.
    std::unique_ptr<Tracker> ptr = std::make_unique<Tracker>("SmartPtr");

    // Usage: Use it exactly like a raw pointer (-> and * operators are overloaded)
    ptr->say_hello();
    
    // NO delete needed here.
}
std::cout << "--- Scope End ---" << std::endl;
// You should see the [Destructed] message BEFORE "Scope End" prints.


--- Scope Start ---
[Constructed] SmartPtr
Hello from SmartPtr
[Destructed ] SmartPtr (Memory Freed)
--- Scope End ---


@0x7851f5a7b5a0

### ⚠️ Crucial: You cannot copy a unique_ptr
Since it represents exclusive ownership, copying it is forbidden (compilation error). This prevents the "Double Free" bug common in C.

If you want to pass it to another function, you must Move it (transfer ownership).


In [3]:
// Function defined globally in the notebook
void take_ownership(std::unique_ptr<Tracker> t) {
    std::cout << "I now own the pointer!" << std::endl;
} // 't' dies here, so the Tracker is destroyed HERE.


In [ ]:
// Test the function
{
    auto ptr = std::make_unique<Tracker>("Mover");
    
    // take_ownership(ptr); // ERROR: Call to implicitly-deleted copy constructor
    
    // std::move casts 'ptr' to an r-value, allowing ownership transfer.
    // 'ptr' is now empty (nullptr).
    take_ownership(std::move(ptr)); 
    
    if (!ptr) {
        std::cout << "ptr is now empty in local scope" << std::endl;
    }
}

[Constructed] Mover
I now own the pointer!
[Destructed ] Mover (Memory Freed)
ptr is now empty in local scope


---

## 3. std::shared_ptr (Reference Counting)

Sometimes, multiple parts of your code need to check a single resource, and you don't know who will finish last.

* Mechanism: Maintains an atomic reference count. 
* Behavior: When you copy a shared_ptr, count increments. When one is destroyed, count decrements. When count hits 0, resource is freed.
* Overhead: Slight (due to atomic counter/thread safety).

Use std::make_shared to create them.


In [5]:
// Shared Pointer Demo
// Create a shared pointer
std::shared_ptr<Tracker> shared1 = std::make_shared<Tracker>("Shared");

{
    // Create a second reference. Count becomes 2.
    std::shared_ptr<Tracker> shared2 = shared1;
    std::cout << "Inside block, use count: " << shared1.use_count() << std::endl;
    
    shared2->say_hello();
}
// shared2 dies. Count becomes 1. Object is NOT destroyed yet.

std::cout << "Outside block, use count: " << shared1.use_count() << std::endl;


[Constructed] Shared
Inside block, use count: 2
Hello from Shared
Outside block, use count: 1


## 4. Summary: The Modern C++ Memory Rules

1.  Never use malloc/free. They don't call Constructors/Destructors.
2.  Avoid new/delete unless you are writing a low-level data structure library.
3.  Default to std::unique_ptr. It is fast and safe.
4.  Use std::shared_ptr only when ownership is truly shared (like nodes in a graph).
5.  Use Raw Pointers (T*) only for observing data (i.e., "I need to look at this, but I don't own it").
